In [9]:
import cv2
import numpy as np
import random as rng
from ipynb.fs.full.wavelet import Wavelet

In [10]:
def resize(frame,name=None):
    scale_percent = 60 # percent of original size
    width = int(frame.shape[1] * scale_percent / 100)
    height = int(frame.shape[0] * scale_percent / 100)
    dim = (width, height)

    # resize image
    resized = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
    
    if name is not None:
        cv2.imshow(name, display)
        
    return resized

######################################################################################################

def drawContour(mask,kernelsize=15):
    
    kernel6 = np.ones((kernelsize,kernelsize), np.uint8)
    dil = cv2.dilate(mask, kernel6, iterations=1)
    contours, _ = cv2.findContours(dil, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    return dil,contours


######################################################################################################

def convert(img, target_type_min, target_type_max, target_type):
    imin = img.min()
    imax = img.max()

    a = (target_type_max - target_type_min) / (imax - imin)
    b = target_type_max - a * imax
    new_img = (a * img + b).astype(target_type)
    
    return new_img

######################################################################################################

def correlation_coefficient(patch1, patch2):
    product = np.mean((patch1 - patch1.mean()) * (patch2 - patch2.mean()))
    stds = patch1.std() * patch2.std()
    if stds == 0:
        return 0
    else:
        product /= stds
        return product
    
######################################################################################################
    
def boundingBoxes(frame_diff,contours,show=False):
    
    contours_poly = [None]*len(contours)
    boundRect = [None]*len(contours)
    
    for i, c in enumerate(contours):
        contours_poly[i] = cv2.approxPolyDP(c, 5, True)
        boundRect[i] = cv2.boundingRect(contours_poly[i])
    
    drawing = np.zeros((frame_diff.shape[0], frame_diff.shape[1], 3), dtype=np.uint8)
    
    for i in range(len(contours)):
        color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
        cv2.drawContours(drawing, contours_poly, i, color)
        cv2.rectangle(drawing, (int(boundRect[i][0]), int(boundRect[i][1])), \
          (int(boundRect[i][0]+boundRect[i][2]), int(boundRect[i][1]+boundRect[i][3])), color, 2)
    
    if show:
        display=resize(drawing)
        cv2.imshow('all boxes', display)

    return frame_diff, boundRect

######################################################################################################

def checkBoxes(frame_diff, boundingBoxes ,show=False,coeff=150,wave_temp=None,wave_temp2=None):
    
    if wave_temp is None:
        print("error no template checkBoxes ")
        return None
    if wave_temp2 is None:
        wave_temp2=wave_temp
    
    size_min=wave_temp.shape[0]//2
    size_max=wave_temp.shape[0]*3
    
    maxBox=None 

    for box in boundingBoxes:

        if box[3]>size_min and box[3]<size_max and box[2]>size_min and box[2]<size_max and box[2]/box[3]<100 and box[2]/box[3]>0.001:

            box_cp = frame_diff[box[1]:(box[1]+box[3]),box[0]:(box[0]+box[2]),:]
            wave_box=Wavelet(box_cp)

            diff1=cv2.absdiff(wave_temp[0:2,0:2,:],wave_box[0:2,0:2,:])#cv2.absdiff(wave_temp[0,0,:],wave_box[0,0,:])+cv2.absdiff(wave_temp[1,1,:],wave_box[1,1,:])
            diff2=cv2.absdiff(wave_temp2[0:2,0:2,:],wave_box[0:2,0:2,:])#cv2.absdiff(wave_temp2[0,0,:],wave_box[0,0,:])+cv2.absdiff(wave_temp2[1,1,:],wave_box[1,1,:])
            diff=min(np.sum(diff1),np.sum(diff2))
        
            diff=np.sum(diff)
            #diff=np.sum(diff,axis=0)
            
            diff_weight=( diff/3 )
            
            if diff_weight is not None and coeff>diff_weight:
                coeff=diff_weight
                maxBox=box

    if maxBox is None:
        return None,None
    
    print(coeff)
    if show:
        box_cp= frame_diff[maxBox[1]:(maxBox[1]+maxBox[3]),maxBox[0]:(maxBox[0]+maxBox[2]),:]   
        cv2.rectangle(frame_diff, (int(box[0]), int(box[1])),(int(box[0]+box[2]), int(box[1]+box[3])), (255,255,255), 2)        
        display = resize(frame_diff)
        cv2.imshow('rec',display)
    
    return frame_diff,maxBox
    
